In [1]:
using AcausalNets
using QI
using LightGraphs
using LinearAlgebra

┌ Info: Recompiling stale cache file /home/marcin/.julia/compiled/v0.7/AcausalNets/sM5bh.ji for AcausalNets [f45f996a-9c6d-58c4-b63c-9e6370055198]
└ @ Base loading.jl:1185
┌ Warning: Package AcausalNets does not have QI in its dependencies:
│ - If you have AcausalNets checked out for development and have
│   added QI as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AcausalNets
└ Loading QI into AcausalNets from project dependency, future warnings for AcausalNets are suppressed.


In [2]:
roA = Diagonal([.5, .5])
roBwA = Diagonal([.5, .5, .4, .6])
roCwA = Diagonal([.7, .3, .2, .8])
roDwB = Diagonal([.9, .1, .5, .5])
roEwC = Diagonal([.3, .7, .6, .4])
roFwDE = Diagonal([.01, .99, .01, .99, .01, .99, .99, .01])
roGwC = Diagonal([.8, .2, .1, .9])
roHwEG = Diagonal([.05, .95, .95, .05, .95, .05, .95, .05])

var_a = Variable(:a, 2)
var_b = Variable(:b, 2)
var_c = Variable(:c, 2)
var_d = Variable(:d, 2)
var_e = Variable(:e, 2)
var_f = Variable(:f, 2)
var_g = Variable(:g, 2)
var_h = Variable(:h, 2)

sys_a = DiscreteQuantumSystem([var_a], roA)
sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
sys_c = DiscreteQuantumSystem([var_a], [var_c], roCwA)
sys_d = DiscreteQuantumSystem([var_b], [var_d], roDwB)
sys_e = DiscreteQuantumSystem([var_c], [var_e], roEwC)
sys_f = DiscreteQuantumSystem([var_d, var_e], [var_f], roFwDE)
sys_g = DiscreteQuantumSystem([var_c], [var_g], roGwC)
sys_h = DiscreteQuantumSystem([var_e, var_g], [var_h], roHwEG)


example_an = AcausalNet()
push!(example_an, sys_a)
push!(example_an, sys_b)
push!(example_an, sys_c)
push!(example_an, sys_d)
push!(example_an, sys_e)
push!(example_an, sys_f)
push!(example_an, sys_g)
push!(example_an, sys_h)

# gplot(example_bn.dag, nodelabel=nodes(example_bn))
# [v.target for v in example_bn.cpds]

AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}({8, 9} directed simple Int64 graph, AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}[DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:a, 2)], [0.5 0.0; 0.0 0.5]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:b, 2)], [0.5 0.0 0.0 0.0; 0.0 0.5 0.0 0.0; 0.0 0.0 0.4 0.0; 0.0 0.0 0.0 0.6]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:c, 2)], [0.7 0.0 0.0 0.0; 0.0 0.3 0.0 0.0; 0.0 0.0 0.2 0.0; 0.0 0.0 0.0 0.8]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:b, 2)], Variable[Variable(:d, 2)], [0.9 0.0 0.0 0.0; 0.0 0.1 0.0 0.0; 0.0 0.0 0.5 0.0; 0.0 0.0 0.0 0.5]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{Abstrac

In [3]:
an = example_an;

In [4]:
import AcausalNets.Inference: moral_graph
mg = moral_graph(an)
println(variables_names(an))
mg.fadjlist

Symbol[:a, :b, :c, :d, :e, :f, :g, :h]


8-element Array{Array{Int64,1},1}:
 [2, 3]         
 [1, 4]         
 [1, 5, 7]      
 [2, 5, 6]      
 [3, 4, 6, 7, 8]
 [4, 5]         
 [3, 5, 8]      
 [5, 7]         

In [5]:
import AcausalNets.Inference: triangulate
triangulated, cliques = triangulate(mg, an)

println(variables_names(an))
triangulated.fadjlist

Symbol[:a, :b, :c, :d, :e, :f, :g, :h]


8-element Array{Array{Int64,1},1}:
 [2, 3]         
 [1, 3, 4]      
 [1, 2, 4, 5, 7]
 [2, 3, 5, 6]   
 [3, 4, 6, 7, 8]
 [4, 5]         
 [3, 5, 8]      
 [5, 7]         

In [6]:
import AcausalNets.Inference: JoinTree

jt = JoinTree(cliques, an);

LoadError: MethodError: Cannot `convert` an object of type Diagonal{Float64,Array{Float64,1}} to an object of type AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at essentials.jl:154

In [ ]:
import AcausalNets.Inference: initialize

init_jt = initialize(jt, an)

In [ ]:
[[variables(s) for s in c] for c in init_jt.clusters]

In [ ]:
import AcausalNets.Inference: single_message_pass

In [ ]:
# bc from bcd
ptrace(init_jt.vertex_to_num[5], [2,2,2], [3])

In [ ]:
# b from abc
ptrace(init_jt.vertex_to_num[6], [2,2,2], [1])

In [ ]:
passed = single_message_pass(5, 6, init_jt, an)
passed_2 = single_message_pass(6, 5, passed, an);

ptrace(passed_2.vertex_to_num[5], [2,2,2], [3])

In [ ]:
ptrace(passed_2.vertex_to_num[6], [2,2,2], [1])
# bc in abc and bcd are now consistent

In [ ]:
import AcausalNets.Inference: global_propagation

propagated_jt = global_propagation(init_jt, an)

In [ ]:
a = 1
a_oper = (ketbra(a,a,2) + ketbra(0,0,2)) / 2
assignments = Dict([
#         var_a => a_oper
        ])

initialized_jt = initialize(jt, an)
propagated_jt = global_propagation(initialized_jt, an)
propagated_jt.clusters

aBC = deepcopy(propagated_jt.vertex_to_num[6])

In [ ]:
aBC = aBC / tr(aBC)

In [ ]:
ptrace(aBC, [2,2,2], [2,3])

In [ ]:
xD = Dict(:a => 1, :b => 2)

In [ ]:
k = keys(xD);

In [ ]:
for key in k
    println(key)
end

In [ ]:
length(xD)